In [1]:
# Step 1: Install kaggle library
!pip install -q kaggle

# Step 2: Upload kaggle.json (API token file)
from google.colab import files
files.upload()  # ← This will prompt you to upload kaggle.json

# Step 3: Move kaggle.json to the correct location
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Step 4: Download the dataset
!kaggle datasets download -d paramaggarwal/fashion-product-images-small

# Step 5: Unzip the dataset
!unzip -q fashion-product-images-small.zip

# Step 6: Load styles.csv using pandas
import pandas as pd

styles_df = pd.read_csv("styles.csv", on_bad_lines='skip')
print("Shape:", styles_df.shape)
styles_df.head()


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small
License(s): MIT
Shape: (44424, 10)


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [2]:
import pandas as pd
import os

# Load styles.csv
styles_df = pd.read_csv("styles.csv", on_bad_lines='skip')

# Drop rows with missing image links or season
styles_df = styles_df.dropna(subset=['id', 'season'])

# Convert id to string and append .jpg to match filenames in the images folder
styles_df['image'] = styles_df['id'].astype(str) + ".jpg"

# Create a dictionary: image filename → season
image_label_dict = dict(zip(styles_df['image'], styles_df['season']))

# Optional: Show a few mappings
for i, (img, label) in enumerate(image_label_dict.items()):
    print(f"{img} → {label}")
    if i == 4:
        break

15970.jpg → Fall
39386.jpg → Summer
59263.jpg → Winter
21379.jpg → Fall
53759.jpg → Summer


In [3]:
# Get unique season labels
unique_labels = styles_df['season'].dropna().unique()

# Count of unique labels
num_labels = len(unique_labels)

print(f"Number of unique season labels: {num_labels}")
print("Labels:", unique_labels)

Number of unique season labels: 4
Labels: ['Fall' 'Summer' 'Winter' 'Spring']


In [4]:
import os
import shutil
from tqdm import tqdm
import pandas as pd


filtered_df = styles_df[styles_df['season'].notna()].copy()


root_dir = "fashion_data_season"
os.makedirs(root_dir, exist_ok=True)


season_labels = filtered_df['season'].astype(str).unique()
for label in season_labels:
    os.makedirs(os.path.join(root_dir, label), exist_ok=True)


source_dir = "images"
for idx, row in tqdm(filtered_df.iterrows(), total=len(filtered_df)):
    img_name = row['image']
    label = str(row['season'])

    src = os.path.join(source_dir, img_name)
    dst = os.path.join(root_dir, label, img_name)

    if os.path.exists(src):
        try:
            shutil.copy(src, dst)
        except Exception as e:
            print(f"❌ Error copying {img_name}: {e}")
    else:
        print(f"⚠️ Image not found: {src}")


 16%|█▋        | 7247/44403 [00:02<00:12, 2881.31it/s]

⚠️ Image not found: images/39403.jpg


 37%|███▋      | 16366/44403 [00:07<00:13, 2034.91it/s]

⚠️ Image not found: images/39410.jpg


 74%|███████▎  | 32637/44403 [00:14<00:04, 2421.50it/s]

...

 83%|████████▎ | 36676/44403 [00:16<00:03, 1946.00it/s]

⚠️ Image not found: images/39425.jpg


 91%|█████████ | 40280/44403 [00:18<00:02, 1901.40it/s]

⚠️ Image not found: images/12347.jpg


100%|██████████| 44403/44403 [00:21<00:00, 2077.78it/s]


In [5]:
import torch
import torch.nn as nn
import timm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load EfficientNet with 4 output classes
model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=4)
model = model.to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

In [6]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


full_dataset = datasets.ImageFolder("/content/fashion_data_season", transform=transform)


total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size


train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [8]:
import torch
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=10, class_names=None):
    best_f1 = 0.0

    for epoch in range(epochs):
        # ---- Training ----
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        f1 = f1_score(all_labels, all_preds, average='weighted')

        # ---- Save Best Model ----
        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), "best_model.pth")
            print(f"✅ Best model updated at Epoch {epoch+1} | F1: {f1:.4f}")

        # ---- Confusion Matrix ----
        cm = confusion_matrix(all_labels, all_preds)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_names if class_names else np.unique(all_labels),
                    yticklabels=class_names if class_names else np.unique(all_labels))
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.title(f'Confusion Matrix - Epoch {epoch+1}')
        plt.tight_layout()
        plt.savefig(f'confusion_matrix_epoch_{epoch+1}.png')
        plt.close()

        # ---- Logging ----
        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | "
              f"Val Loss: {avg_val_loss:.4f} | F1 Score: {f1:.4f}")



In [9]:
train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    epochs=10
)


✅ Best model updated at Epoch 1 | F1: 0.7064
Epoch 1 | Train Loss: 0.9278 | Val Loss: 0.6885 | F1 Score: 0.7064
✅ Best model updated at Epoch 2 | F1: 0.7450
Epoch 2 | Train Loss: 0.6070 | Val Loss: 0.6434 | F1 Score: 0.7450
✅ Best model updated at Epoch 3 | F1: 0.7676
Epoch 3 | Train Loss: 0.5030 | Val Loss: 0.6123 | F1 Score: 0.7676
✅ Best model updated at Epoch 4 | F1: 0.7774
Epoch 4 | Train Loss: 0.3933 | Val Loss: 0.6276 | F1 Score: 0.7774
Epoch 5 | Train Loss: 0.2886 | Val Loss: 0.7096 | F1 Score: 0.7695
Epoch 6 | Train Loss: 0.2102 | Val Loss: 0.7704 | F1 Score: 0.7741
Epoch 7 | Train Loss: 0.1519 | Val Loss: 0.8929 | F1 Score: 0.7512
Epoch 8 | Train Loss: 0.1235 | Val Loss: 0.8601 | F1 Score: 0.7673
Epoch 9 | Train Loss: 0.1033 | Val Loss: 0.9273 | F1 Score: 0.7754
Epoch 10 | Train Loss: 0.0915 | Val Loss: 0.9415 | F1 Score: 0.7705


In [10]:
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F
import copy

In [11]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='macro')
print(f"\n📊 Test Accuracy: {acc:.4f}, F1 (macro): {f1:.4f}")


📊 Test Accuracy: 0.7766, F1 (macro): 0.7737


In [12]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')
print(f"\n📊 Test Accuracy: {acc:.4f}, F1 (weighted): {f1:.4f}")


📊 Test Accuracy: 0.7766, F1 (weighted): 0.7753
